In [ ]:
pip install langchain

In [ ]:
pip install langchain_community

In [ ]:
import asyncio
import json
import logging
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass
from pathlib import Path
import hashlib
import numpy as np

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.vectorstores.base import VectorStore
from langchain.callbacks.manager import CallbackManagerForRetrieverRun

In [ ]:
# Install core dependencies
!pip install -U sentence-transformers
!pip install transformers torch
!pip install numpy

# Install MCP (Model Context Protocol)
!pip install mcp

# Install Ollama Python client
!pip install ollama

# Additional dependencies
!pip install asyncio-mqtt
!pip install python-dotenv

In [ ]:
pip install chromadb langchain sentence-transformers ollama mcp

In [ ]:
# Install Ollama (Linux/Mac)
!curl -fsSL https://ollama.ai/install.sh | sh

# Or for Windows, download from: https://ollama.ai/download

# Pull LLAMA3 model
!ollama pull llama3

# Verify installation
!ollama list

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Error: ollama server not responding - could not connect to ollama server, run 'ollama serve' to start it
Error: ollama server not responding - could not connect to ollama server, run 'ollama serve' to start it


In [ ]:
import asyncio
import json
import logging
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass
from pathlib import Path
import hashlib
import numpy as np
import uuid


In [ ]:
try:
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.embeddings import HuggingFaceEmbeddings
    from langchain.schema import Document
    from langchain.llms import Ollama
    from langchain.chains import RetrievalQA
    from langchain.vectorstores.base import VectorStore
    from langchain.callbacks.manager import CallbackManagerForRetrieverRun
except ImportError as e:
    print(f"LangChain import error: {e}")
    print("Install with: pip install langchain langchain-community")


In [ ]:
try:
    import chromadb
    from chromadb.config import Settings
    from chromadb.utils import embedding_functions
except ImportError as e:
    print(f"ChromaDB import error: {e}")
    print("Install with: pip install chromadb")


In [ ]:
try:
    from mcp.server import Server
    from mcp.types import Resource, Tool
    import mcp.server.stdio
except ImportError as e:
    print(f"MCP import error: {e}")
    print("Install with: pip install mcp")

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [ ]:
from dataclasses import dataclass

@dataclass
class ClassifiedDocument:
    """Represents a document"""
    content: str
    metadata: Dict[str, Any]
    source: str

In [ ]:
from dataclasses import dataclass

@dataclass
class EmbeddedChunk:
    """Represents a document chunk with embedding"""
    id: str
    content: str
    embedding: List[float]
    metadata: Dict[str, Any]
    parent_doc_id: str

In [ ]:
class DocumentProcessor:
    """Handles document parsing and chunking"""

    def __init__(self, chunk_size: int = 1000, chunk_overlap: int = 200):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", ". ", " ", ""]
        )

    def parse_document(self, file_path: str) -> ClassifiedDocument:
        """Parse document from file path"""
        path = Path(file_path)

        if not path.exists():
            raise FileNotFoundError(f"File not found: {file_path}")

        # Handle different file types
        content = ""
        if path.suffix.lower() == '.txt':
            content = path.read_text(encoding='utf-8')
        elif path.suffix.lower() == '.json':
            data = json.loads(path.read_text(encoding='utf-8'))
            content = json.dumps(data, indent=2)
        elif path.suffix.lower() in ['.md', '.markdown']:
            content = path.read_text(encoding='utf-8')
        elif path.suffix.lower() == '.csv':
            import pandas as pd
            df = pd.read_csv(path)
            content = df.to_string()
        else:
            # For other formats, try to read as text
            try:
                content = path.read_text(encoding='utf-8')
            except UnicodeDecodeError:
                content = path.read_text(encoding='latin-1')

        metadata = {
            "file_name": path.name,
            "file_path": str(path),
            "file_size": path.stat().st_size,
            "file_type": path.suffix.lower()
        }

        return ClassifiedDocument(
            content=content,
            metadata=metadata,
            source=file_path
        )

    def chunk_document(self, doc: ClassifiedDocument) -> List[Dict[str, Any]]:
        """Split document into chunks"""
        chunks = self.text_splitter.split_text(doc.content)

        chunked_docs = []
        for i, chunk in enumerate(chunks):
            chunk_id = hashlib.md5(f"{doc.source}_{i}_{chunk[:100]}".encode()).hexdigest()

            chunked_docs.append({
                "id": chunk_id,
                "content": chunk,
                "chunk_index": i,
                "parent_doc_id": hashlib.md5(doc.source.encode()).hexdigest(),
                "metadata": {
                    **doc.metadata,
                    "chunk_size": len(chunk),
                    "total_chunks": len(chunks)
                }
            })

        return chunked_docs

In [ ]:
class ClasswiseEmbedder:
    """Handles embedding generation""" # Updated description

    def __init__(self, model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
        self.embeddings = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs={'device': 'cpu'},
            encode_kwargs={'normalize_embeddings': True}
        )
        # self.class_embedders = {}  # Removed class-wise cache

    def embed_chunks(self, chunks: List[Dict[str, Any]]) -> List[EmbeddedChunk]:
        """Generate embeddings for chunks""" # Updated description
        embedded_chunks = []

        # Extract content for batch embedding
        texts = [chunk["content"] for chunk in chunks]

        # Generate embeddings in batch
        embeddings = self.embeddings.embed_documents(texts)

        # Create embedded chunks
        for chunk, embedding in zip(chunks, embeddings):
            embedded_chunk = EmbeddedChunk(
                id=chunk["id"],
                content=chunk["content"],
                embedding=embedding,
                # classification=classification, # Removed classification
                metadata=chunk["metadata"],
                parent_doc_id=chunk["parent_doc_id"]
            )
            embedded_chunks.append(embedded_chunk)

        return embedded_chunks

In [ ]:
class ChromaVectorStore:
    """ChromaDB-based vector store""" # Updated description

    def __init__(self, persist_directory: str = "./chroma_db", collection_name: str = "rag_documents"):
        self.persist_directory = persist_directory
        self.collection_name = collection_name
        self.client = None
        self.collection = None
        self.embedding_function = None # This might be needed later if not using LangChain's embedder

    def initialize(self):
        """Initialize ChromaDB client and collection"""
        # Create ChromaDB client with persistence
        settings = Settings(
            persist_directory=self.persist_directory,
            anonymized_telemetry=False
        )

        self.client = chromadb.PersistentClient(
            path=self.persist_directory,
            settings=settings
        )

        # Create or get collection
        try:
            self.collection = self.client.get_collection(name=self.collection_name)
            logger.info(f"Using existing collection: {self.collection_name}")
        except:
            self.collection = self.client.create_collection(
                name=self.collection_name,
                metadata={"hnsw:space": "cosine"}
            )
            logger.info(f"Created new collection: {self.collection_name}")

        logger.info("ChromaDB initialized successfully")

    def add_embeddings(self, embedded_chunks: List[EmbeddedChunk]):
        """Store embedded chunks in ChromaDB"""
        if not self.collection:
            raise ValueError("ChromaDB not initialized. Call initialize() first.")

        # Prepare data for ChromaDB
        ids = []
        embeddings = []
        documents = []
        metadatas = []

        for chunk in embedded_chunks:
            ids.append(chunk.id)
            embeddings.append(chunk.embedding)
            documents.append(chunk.content)

            # Combine all metadata
            metadata = {
                "parent_doc_id": chunk.parent_doc_id,
                **chunk.metadata
            }
            metadatas.append(metadata)

        # Add to collection in batches (ChromaDB has batch size limits)
        batch_size = 1000
        for i in range(0, len(ids), batch_size):
            batch_ids = ids[i:i+batch_size]
            batch_embeddings = embeddings[i:i+batch_size]
            batch_documents = documents[i:i+batch_size]
            batch_metadatas = metadatas[i:i+batch_size]

            self.collection.add(
                ids=batch_ids,
                embeddings=batch_embeddings,
                documents=batch_documents,
                metadatas=batch_metadatas
            )

        logger.info(f"Stored {len(embedded_chunks)} embedded chunks in ChromaDB")

    def similarity_search(self, query: str, k: int = 5) -> List[Dict[str, Any]]:
        """Perform similarity search""" # Updated description
        if not self.collection:
            raise ValueError("ChromaDB not initialized. Call initialize() first.")

        # Generate query embedding
        embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        query_embedding = embedder.embed_query(query)

        # Query ChromaDB
        results = self.collection.query(
            query_embeddings=[query_embedding],
            n_results=k,
            include=["documents", "metadatas", "distances"]
        )

        # Format results
        formatted_results = []
        if results['documents'] and len(results['documents'][0]) > 0:
            for i, (doc, metadata, distance) in enumerate(zip(
                results['documents'][0],
                results['metadatas'][0],
                results['distances'][0]
            )):
                formatted_results.append({
                    "id": results['ids'][0][i],
                    "content": doc,
                    "metadata": metadata,
                    "parent_doc_id": metadata.get("parent_doc_id", "unknown"),
                    "similarity": 1 - distance,  # Convert distance to similarity
                    "distance": distance
                })

        return formatted_results

    def get_collection_stats(self) -> Dict[str, Any]:
        """Get collection statistics"""
        if not self.collection:
            return {"error": "Collection not initialized"}

        count = self.collection.count()

        return {
            "total_documents": count,
        }

    def delete_documents(self, parent_doc_id: Optional[str] = None) -> Dict[str, Any]:
        """Delete documents by parent document ID""" # Updated description
        try:
            if not self.collection:
                return {"error": "ChromaDB not initialized"}

            where_clause = {}
            if parent_doc_id:
                where_clause["parent_doc_id"] = {"$eq": parent_doc_id}

            if not where_clause:
                return {"error": "Must specify parent_doc_id"} # Updated error message

            # Get documents to delete
            to_delete = self.collection.get(
                where=where_clause,
                include=["metadatas"]
            )

            if to_delete['ids']:
                self.collection.delete(ids=to_delete['ids'])
                return {"status": "success", "deleted_count": len(to_delete['ids'])}
            else:
                return {"status": "success", "deleted_count": 0, "message": "No documents found matching criteria"}

        except Exception as e:
            logger.error(f"Error deleting documents: {str(e)}")
            return {"error": str(e)}

In [ ]:
class ChromaRetriever:
    """Custom retriever for LangChain integration with ChromaDB"""

    def __init__(self, vector_store: ChromaVectorStore, k: int = 5):
        self.vector_store = vector_store
        self.k = k
        # self.classification_filter = classification_filter # Removed classification_filter

    def get_relevant_documents(self, query: str) -> List[Document]:
        """Retrieve relevant documents"""
        results = self.vector_store.similarity_search(
            query=query,
            k=self.k,
            # classification_filter=self.classification_filter # Removed classification_filter
        )

        documents = []
        for result in results:
            doc = Document(
                page_content=result["content"],
                metadata={
                    **result["metadata"],
                    # "classification": result["classification"], # Removed classification
                    "similarity": result["similarity"],
                    "parent_doc_id": result["parent_doc_id"]
                }
            )
            documents.append(doc)

        return documents

In [ ]:
class RAGSystem:
    """Complete RAG system orchestrator with ChromaDB"""

    def __init__(self, persist_directory: str = "./chroma_db", collection_name: str = "rag_documents"):
        self.processor = DocumentProcessor()
        self.embedder = ClasswiseEmbedder()
        self.vector_store = ChromaVectorStore(persist_directory, collection_name)
        self.llm = Ollama(model="llama3")
        self.retriever = None
        self.qa_chain = None

    def initialize(self):
        """Initialize the RAG system"""
        self.vector_store.initialize()
        logger.info("RAG system initialized with ChromaDB")

    def ingest_document(self, file_path: str) -> Dict[str, Any]:
        """Complete document ingestion pipeline"""
        try:
            # Parse document
            doc = self.processor.parse_document(file_path)

            # Chunk document
            chunks = self.processor.chunk_document(doc)

            # Generate embeddings
            embedded_chunks = self.embedder.embed_chunks(chunks)

            # Store in vector database
            self.vector_store.add_embeddings(embedded_chunks)

            logger.info(f"Successfully ingested document: {file_path}")
            return {"status": "success", "chunks_created": len(chunks)}

        except Exception as e:
            logger.error(f"Error ingesting document {file_path}: {str(e)}")
            return {"status": "error", "message": str(e)}

    def query(self, question: str, k: int = 5) -> Dict[str, Any]:
        """Query the RAG system"""
        try:
            # Setup retriever
            retriever = ChromaRetriever(
                vector_store=self.vector_store,
                k=k,
            )

            # Get relevant documents
            docs = retriever.get_relevant_documents(question)

            if not docs:
                return {
                    "answer": "No relevant documents found.",
                    "sources": [],
                }

            # Create context from retrieved documents
            context = "\n\n".join([doc.page_content for doc in docs])

            # Generate answer using LLAMA3
            prompt = f"""Based on the following context, answer the question. If the answer cannot be found in the context, say so clearly.

Context:
{context}

Question: {question}

Answer:"""

            answer = self.llm(prompt)

            # Prepare sources
            sources = []
            for doc in docs:
                sources.append({
                    "content": doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content,
                    "similarity": doc.metadata.get("similarity"),
                    "file_name": doc.metadata.get("file_name")
                })

            return {
                "answer": answer,
                "sources": sources,
                "num_sources": len(sources)
            }

        except Exception as e:
            logger.error(f"Error querying RAG system: {str(e)}")
            return {"error": str(e)}

    def get_stats(self) -> Dict[str, Any]:
        """Get system statistics"""
        try:
            return self.vector_store.get_collection_stats()
        except Exception as e:
            logger.error(f"Error getting stats: {str(e)}")
            return {"error": str(e)}

    def delete_documents(self, parent_doc_id: Optional[str] = None) -> Dict[str, Any]:
        """Delete documents by parent document ID"""
        try:
            if not self.vector_store.collection:
                return {"error": "ChromaDB not initialized"}

            where_clause = {}
            if parent_doc_id:
                where_clause["parent_doc_id"] = {"$eq": parent_doc_id}

            if not where_clause:
                return {"error": "Must specify parent_doc_id"}

            # Get documents to delete
            to_delete = self.vector_store.collection.get(
                where=where_clause,
                include=["metadatas"]
            )

            if to_delete['ids']:
                self.vector_store.collection.delete(ids=to_delete['ids'])
                return {"status": "success", "deleted_count": len(to_delete['ids'])}
            else:
                return {"status": "success", "deleted_count": 0, "message": "No documents found matching criteria"}

        except Exception as e:
            logger.error(f"Error deleting documents: {str(e)}")
            return {"error": str(e)}

In [ ]:
class MCPRAGServer:
    """MCP Server for the RAG system with ChromaDB"""

    def __init__(self, persist_directory: str = "./chroma_db"):
        self.rag_system = RAGSystem(persist_directory=persist_directory)
        self.server = Server("rag-chroma-server")
        self._setup_handlers()

    def _setup_handlers(self):
        """Setup MCP server handlers"""

        @self.server.list_resources()
        def list_resources() -> List[Resource]:
            """List available resources"""
            return [
                Resource(
                    uri="rag://stats",
                    name="RAG System Statistics",
                    mimeType="application/json",
                ),
            ]

        @self.server.read_resource()
        def read_resource(uri: str) -> str:
            """Read resource content"""
            if uri == "rag://stats":
                stats = self.rag_system.get_stats()
                return json.dumps(stats, indent=2)
            else:
                raise ValueError(f"Unknown resource: {uri}")

        @self.server.list_tools()
        def list_tools() -> List[Tool]:
            """List available tools"""
            return [
                Tool(
                    name="ingest_document",
                    description="Ingest a document into the RAG system",
                    inputSchema={
                        "type": "object",
                        "properties": {
                            "file_path": {"type": "string", "description": "Path to the document file"},
                        },
                        "required": ["file_path"]
                    }
                ),
                Tool(
                    name="query_rag",
                    description="Query the RAG system",
                    inputSchema={
                        "type": "object",
                        "properties": {
                            "question": {"type": "string", "description": "Question to ask"},
                            "k": {"type": "integer", "description": "Number of documents to retrieve", "default": 5}
                        },
                        "required": ["question"]
                    }
                ),
                Tool(
                    name="get_system_stats",
                    description="Get RAG system statistics",
                    inputSchema={
                        "type": "object",
                        "properties": {}
                    }
                ),
                Tool(
                    name="delete_documents",
                    description="Delete documents by parent document ID",
                    inputSchema={
                        "type": "object",
                        "properties": {
                            "parent_doc_id": {"type": "string", "description": "Delete all chunks of this parent document"}
                        }
                    }
                ),
            ]

        @self.server.call_tool()
        def call_tool(name: str, arguments: dict) -> List[Any]:
            """Handle tool calls"""
            if name == "ingest_document":
                result = self.rag_system.ingest_document(
                    file_path=arguments["file_path"],
                )
                return [{"result": result}]

            elif name == "query_rag":
                result = self.rag_system.query(
                    question=arguments["question"],
                    k=arguments.get("k", 5)
                )
                return [{"result": result}]

            elif name == "get_system_stats":
                result = self.rag_system.get_stats()
                return [{"result": result}]

            elif name == "delete_documents":
                result = self.rag_system.delete_documents(
                    parent_doc_id=arguments.get("parent_doc_id")
                )
                return [{"result": result}]

            else:
                raise ValueError(f"Unknown tool: {name}")

    def run(self):
        """Run the MCP server"""
        self.rag_system.initialize()

        # For synchronous operation in environments that don't support async MCP
        import mcp.server.stdio as stdio
        stdio.run_server(self.server)

In [ ]:

# Utility functions for Jupyter notebook usage
def create_sample_documents():
    """Create sample documents for testing"""
    sample_docs = [
        {
            "filename": "tech_doc.txt",
            "content": "Machine learning is a subset of artificial intelligence that enables computers to learn and make decisions without being explicitly programmed. It uses algorithms to analyze data, identify patterns, and make predictions or classifications.",
            "classification": "technical",
            "confidence": 0.95
        },
        {
            "filename": "business_doc.txt",
            "content": "Our quarterly revenue has increased by 15% compared to last year. The marketing team has successfully launched three new campaigns, resulting in improved customer acquisition and retention rates.",
            "classification": "business",
            "confidence": 0.88
        },
        {
            "filename": "research_doc.txt",
            "content": "The study examined the effects of climate change on marine ecosystems. Results showed a significant correlation between rising sea temperatures and coral bleaching events across multiple reef systems.",
            "classification": "research",
            "confidence": 0.92
        }
    ]

    # Create sample files
    for doc in sample_docs:
        with open(doc["filename"], "w") as f:
            f.write(doc["content"])

    return sample_docs


In [ ]:
# Example usage and testing
def main():
    """Example usage of the RAG system"""
    # Initialize RAG system
    rag = RAGSystem(persist_directory="./test_chroma_db")
    rag.initialize()

    # Define your actual documents here
    actual_documents = [
        {
            "file_path": "/content/business_doc.txt", # Replace with your actual file path
        },
        {
            "file_path": "/content/engineering_drawing_specifications.pdf", # Replace with your actual file path
        },
        # Add more documents as needed
        {
            "file_path": "/content/maintenance_job_card.pdf", # Replace with your actual file path
        },
        # Add more documents as needed
        {
            "file_path": "/content/research_doc.txt", # Replace with your actual file path
        },
        # Add more documents as needed
        {
            "file_path": "/content/tech_doc.txt", # Replace with your actual file path
        },
        # Add more documents as needed
    ]

    print("Ingesting actual documents...")
    for doc_info in actual_documents:
        result = rag.ingest_document(doc_info["file_path"]) # Removed classification and confidence
        print(f"Ingested {doc_info['file_path']}: {result}")

    # Show stats
    print(f"\nSystem Stats: {rag.get_stats()}")


    # Interactive querying
    print("\n" + "="*50)
    print("Interactive RAG Querying")
    print("="*50)
    print("Enter your questions. Type 'quit' or 'exit' to stop.")

    while True:
        question = input("\nYour question: ")
        if question.lower() in ["quit", "exit"]:
            break

        print("Processing your question...")
        result = rag.query(question)

        print(f"Answer: {result.get('answer', 'No answer')}")
        sources = result.get('sources', [])
        print(f"Sources Found: {len(sources)}")
        if sources:
            print("--- Sources ---")
            for i, source in enumerate(sources):
                print(f"Source {i+1}: {source.get('file_name', 'Unknown File')} (Similarity: {source.get('similarity', 'N/A'):.4f})")
                print(f"Content: {source.get('content', 'N/A')}")
                print("-" * 10)
            print("---------------")


if __name__ == "__main__":
    main()

Ingesting actual documents...
Ingested /content/business_doc.txt: {'status': 'success', 'chunks_created': 1}
Ingested /content/engineering_drawing_specifications.pdf: {'status': 'success', 'chunks_created': 10}
Ingested /content/maintenance_job_card.pdf: {'status': 'success', 'chunks_created': 11}
Ingested /content/research_doc.txt: {'status': 'success', 'chunks_created': 1}
Ingested /content/tech_doc.txt: {'status': 'success', 'chunks_created': 1}

System Stats: {'total_documents': 24}

Interactive RAG Querying
Enter your questions. Type 'quit' or 'exit' to stop.

Your question: quit


# Task
Integrate the provided Python code for interacting with the Ollama API into the existing RAG system codebase to use the `llama3-chatqa` model for generating answers.

## Create a custom ollama llm class

### Subtask:
Define a new Python class `CustomOllamaLLM` that inherits from a Langchain BaseLLM or similar base class. This class will encapsulate the logic for interacting with the Ollama API using the provided `query_ollama_api` or `chat_with_ollama` functions.


**Reasoning**:
Define the `CustomOllamaLLM` class inheriting from `BaseLLM` and include the necessary methods for interacting with the Ollama API.



In [ ]:
from langchain.llms.base import BaseLLM
from typing import Any, List, Mapping, Optional
import requests

async def query_ollama_api(api_url: str, model_name: str, prompt: str) -> str:
    """Queries the Ollama API for a text generation."""
    url = f"{api_url}/api/generate"
    payload = {
        "model": model_name,
        "prompt": prompt,
        "stream": False
    }
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()  # Raise an exception for bad status codes
        result = response.json()
        return result.get("response", "").strip()
    except requests.exceptions.RequestException as e:
        logger.error(f"Error querying Ollama API: {e}")
        return f"Error querying Ollama API: {e}"

# Although the original code included chat_with_ollama, the RAG query logic currently uses a single prompt for the LLM.
# We will adapt the query_ollama_api function for the CustomOllamaLLM to fit the current RAG query structure.
# If chat functionality is required later, we can integrate it.

class CustomOllamaLLM(BaseLLM):
    """Custom Langchain LLM for interacting with the Ollama API."""

    ollama_url: str
    model_name: str = "llama3-chatqa"

    @property
    def _llm_type(self) -> str:
        return "custom_ollama"

    def __init__(self, ollama_url: str = "http://localhost:11434", model_name: str = "llama3-chatqa", **kwargs: Any):
        super().__init__(**kwargs)
        self.ollama_url = ollama_url
        self.model_name = model_name

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """Call the Ollama API to generate text."""
        # Langchain's _call is synchronous, so we'll use the synchronous requests call
        # If async is strictly needed, we would use BaseChatModel and its async methods
        url = f"{self.ollama_url}/api/generate"
        payload = {
            "model": self.model_name,
            "prompt": prompt,
            "stream": False
        }
        try:
            response = requests.post(url, json=payload)
            response.raise_for_status()
            result = response.json()
            return result.get("response", "").strip()
        except requests.exceptions.RequestException as e:
            logger.error(f"Error in CustomOllamaLLM call: {e}")
            return f"Error generating response: {e}"


    @property
    def _identifyingparams(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"ollama_url": self.ollama_url, "model_name": self.model_name}


## Implement the ` call` or ` generate` method

### Subtask:
Within the custom LLM class, implement the method required by the Langchain base class (usually `_call` for simple LLMs or `_generate` for more complex ones) to send the prompt to the Ollama API and return the generated response.


## Integrate the custom llm into ragsystem

### Subtask:
Modify the `RAGSystem` class to use an instance of the new custom Ollama LLM class instead of the existing `Ollama(model="llama3")`.


**Reasoning**:
Modify the RAGSystem class to use the CustomOllamaLLM.



In [ ]:
class RAGSystem:
    """Complete RAG system orchestrator with ChromaDB"""

    def __init__(self, persist_directory: str = "./chroma_db", collection_name: str = "rag_documents"):
        self.processor = DocumentProcessor()
        self.embedder = ClasswiseEmbedder()
        self.vector_store = ChromaVectorStore(persist_directory, collection_name)
        # Use the custom Ollama LLM
        self.llm = CustomOllamaLLM(ollama_url="http://localhost:11434", model_name="llama3-chatqa")
        self.retriever = None
        self.qa_chain = None

    def initialize(self):
        """Initialize the RAG system"""
        self.vector_store.initialize()
        logger.info("RAG system initialized with ChromaDB")

    def ingest_document(self, file_path: str) -> Dict[str, Any]:
        """Complete document ingestion pipeline"""
        try:
            # Parse document
            doc = self.processor.parse_document(file_path)

            # Chunk document
            chunks = self.processor.chunk_document(doc)

            # Generate embeddings
            embedded_chunks = self.embedder.embed_chunks(chunks)

            # Store in vector database
            self.vector_store.add_embeddings(embedded_chunks)

            logger.info(f"Successfully ingested document: {file_path}")
            return {"status": "success", "chunks_created": len(chunks)}

        except Exception as e:
            logger.error(f"Error ingesting document {file_path}: {str(e)}")
            return {"status": "error", "message": str(e)}

    def query(self, question: str, k: int = 5) -> Dict[str, Any]:
        """Query the RAG system"""
        try:
            # Setup retriever
            retriever = ChromaRetriever(
                vector_store=self.vector_store,
                k=k,
            )

            # Get relevant documents
            docs = retriever.get_relevant_documents(question)

            if not docs:
                return {
                    "answer": "No relevant documents found.",
                    "sources": [],
                }

            # Create context from retrieved documents
            context = "\n\n".join([doc.page_content for doc in docs])

            # Generate answer using LLAMA3
            prompt = f"""Based on the following context, answer the question. If the answer cannot be found in the context, say so clearly.

Context:
{context}

Question: {question}

Answer:"""

            answer = self.llm(prompt)

            # Prepare sources
            sources = []
            for doc in docs:
                sources.append({
                    "content": doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content,
                    "similarity": doc.metadata.get("similarity"),
                    "file_name": doc.metadata.get("file_name")
                })

            return {
                "answer": answer,
                "sources": sources,
                "num_sources": len(sources)
            }

        except Exception as e:
            logger.error(f"Error querying RAG system: {str(e)}")
            return {"error": str(e)}

    def get_stats(self) -> Dict[str, Any]:
        """Get system statistics"""
        try:
            return self.vector_store.get_collection_stats()
        except Exception as e:
            logger.error(f"Error getting stats: {str(e)}")
            return {"error": str(e)}

    def delete_documents(self, parent_doc_id: Optional[str] = None) -> Dict[str, Any]:
        """Delete documents by parent document ID"""
        try:
            if not self.vector_store.collection:
                return {"error": "ChromaDB not initialized"}

            where_clause = {}
            if parent_doc_id:
                where_clause["parent_doc_id"] = {"$eq": parent_doc_id}

            if not where_clause:
                return {"error": "Must specify parent_doc_id"}

            # Get documents to delete
            to_delete = self.vector_store.collection.get(
                where=where_clause,
                include=["metadatas"]
            )

            if to_delete['ids']:
                self.vector_store.collection.delete(ids=to_delete['ids'])
                return {"status": "success", "deleted_count": len(to_delete['ids'])}
            else:
                return {"status": "success", "deleted_count": 0, "message": "No documents found matching criteria"}

        except Exception as e:
            logger.error(f"Error deleting documents: {str(e)}")
            return {"error": str(e)}

## Update the `main` function (if necessary)

### Subtask:
Ensure the `main` function correctly initializes and uses the `RAGSystem` with the new custom LLM.


**Reasoning**:
Review the `main` function to confirm it initializes and uses the RAGSystem as expected after the previous modification. No code changes are anticipated.



In [ ]:
# Example usage and testing
def main():
    """Example usage of the RAG system"""
    # Initialize RAG system
    # The RAGSystem class now internally uses CustomOllamaLLM with llama3-chatqa
    rag = RAGSystem(persist_directory="./test_chroma_db")
    rag.initialize()

    # Define your actual documents here
    actual_documents = [
        {
            "file_path": "/content/business_doc.txt", # Replace with your actual file path
        },
        {
            "file_path": "/content/engineering_drawing_specifications.pdf", # Replace with your actual file path
        },
        # Add more documents as needed
        {
            "file_path": "/content/maintenance_job_card.pdf", # Replace with your actual file path
        },
        # Add more documents as needed
        {
            "file_path": "/content/research_doc.txt", # Replace with your actual file path
        },
        # Add more documents as needed
        {
            "file_path": "/content/tech_doc.txt", # Replace with your actual file path
        },
        # Add more documents as needed
    ]

    print("Ingesting actual documents...")
    for doc_info in actual_documents:
        # These calls are compatible with the updated RAGSystem
        result = rag.ingest_document(doc_info["file_path"])
        print(f"Ingested {doc_info['file_path']}: {result}")

    # Show stats
    print(f"\nSystem Stats: {rag.get_stats()}")


    # Interactive querying
    print("\n" + "="*50)
    print("Interactive RAG Querying")
    print("="*50)
    print("Enter your questions. Type 'quit' or 'exit' to stop.")

    while True:
        question = input("\nYour question: ")
        if question.lower() in ["quit", "exit"]:
            break

        print("Processing your question...")
        # These calls are compatible with the updated RAGSystem
        result = rag.query(question)

        print(f"Answer: {result.get('answer', 'No answer')}")
        sources = result.get('sources', [])
        print(f"Sources Found: {len(sources)}")
        if sources:
            print("--- Sources ---")
            for i, source in enumerate(sources):
                print(f"Source {i+1}: {source.get('file_name', 'Unknown File')} (Similarity: {source.get('similarity', 'N/A'):.4f})")
                print(f"Content: {source.get('content', 'N/A')}")
                print("-" * 10)
            print("---------------")


if __name__ == "__main__":
    main()

TypeError: Can't instantiate abstract class CustomOllamaLLM without an implementation for abstract method '_generate'

**Reasoning**:
The error indicates that `CustomOllamaLLM` needs to implement the abstract method `_generate`. I need to modify the `CustomOllamaLLM` class to implement `_generate` instead of or in addition to `_call` to satisfy the requirements of the Langchain `BaseLLM`. I will update the class definition to include the `_generate` method.



In [ ]:
from langchain.llms.base import BaseLLM
from langchain_core.outputs import LLMResult
from typing import Any, List, Mapping, Optional
import requests
import asyncio # Import asyncio for potential async operations if needed later

async def query_ollama_api(api_url: str, model_name: str, prompt: str) -> str:
    """Queries the Ollama API for a text generation."""
    url = f"{api_url}/api/generate"
    payload = {
        "model": model_name,
        "prompt": prompt,
        "stream": False
    }
    try:
        # Use asyncio for awaited operations if the context supports it
        # For now, keep the synchronous requests call within _generate
        # If async is required, this function would need to be used within an async context
        response = requests.post(url, json=payload)
        response.raise_for_status()  # Raise an exception for bad status codes
        result = response.json()
        return result.get("response", "").strip()
    except requests.exceptions.RequestException as e:
        logger.error(f"Error querying Ollama API: {e}")
        return f"Error querying Ollama API: {e}"


class CustomOllamaLLM(BaseLLM):
    """Custom Langchain LLM for interacting with the Ollama API."""

    ollama_url: str
    model_name: str = "llama3-chatqa"

    @property
    def _llm_type(self) -> str:
        return "custom_ollama"

    def __init__(self, ollama_url: str = "http://localhost:11434", model_name: str = "llama3-chatqa", **kwargs: Any):
        super().__init__(**kwargs)
        self.ollama_url = ollama_url
        self.model_name = model_name

    # Implement the _generate method as required by the abstract base class
    def _generate(self, prompts: List[str], stop: Optional[List[str]] = None, run_manager: Optional[Any] = None) -> LLMResult:
        """Generate text based on the provided prompts."""
        # This method is expected to handle a list of prompts and return an LLMResult
        # For simplicity, we'll process each prompt individually for now
        generations = []
        for prompt in prompts:
            text = self._call(prompt, stop=stop) # Reuse the existing _call logic
            generations.append([{"text": text}]) # Langchain expects a list of lists of generations

        return LLMResult(generations=generations)

    # Keep _call for internal use or backward compatibility if needed, though _generate is the primary method
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """Call the Ollama API to generate text (synchronous helper)."""
        url = f"{self.ollama_url}/api/generate"
        payload = {
            "model": self.model_name,
            "prompt": prompt,
            "stream": False
        }
        try:
            response = requests.post(url, json=payload)
            response.raise_for_status()
            result = response.json()
            return result.get("response", "").strip()
        except requests.exceptions.RequestException as e:
            logger.error(f"Error in CustomOllamaLLM call: {e}")
            return f"Error generating response: {e}"

    @property
    def _identifyingparams(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"ollama_url": self.ollama_url, "model_name": self.model_name}


**Reasoning**:
Now that the `CustomOllamaLLM` class has the required `_generate` method, I need to re-run the `main` function code block to verify that the `RAGSystem` can be initialized and used without the `TypeError`.



In [ ]:
# Example usage and testing
def main():
    """Example usage of the RAG system"""
    # Initialize RAG system
    # The RAGSystem class now internally uses CustomOllamaLLM with llama3-chatqa
    rag = RAGSystem(persist_directory="./test_chroma_db")
    rag.initialize()

    # Define your actual documents here
    actual_documents = [
        {
            "file_path": "/content/business_doc.txt", # Replace with your actual file path
        },
        {
            "file_path": "/content/engineering_drawing_specifications.pdf", # Replace with your actual file path
        },
        # Add more documents as needed
        {
            "file_path": "/content/maintenance_job_card.pdf", # Replace with your actual file path
        },
        # Add more documents as needed
        {
            "file_path": "/content/research_doc.txt", # Replace with your actual file path
        },
        # Add more documents as needed
        {
            "file_path": "/content/tech_doc.txt", # Replace with your actual file path
        },
        # Add more documents as needed
    ]

    print("Ingesting actual documents...")
    for doc_info in actual_documents:
        # These calls are compatible with the updated RAGSystem
        result = rag.ingest_document(doc_info["file_path"])
        print(f"Ingested {doc_info['file_path']}: {result}")

    # Show stats
    print(f"\nSystem Stats: {rag.get_stats()}")


    # Interactive querying
    print("\n" + "="*50)
    print("Interactive RAG Querying")
    print("="*50)
    print("Enter your questions. Type 'quit' or 'exit' to stop.")

    while True:
        question = input("\nYour question: ")
        if question.lower() in ["quit", "exit"]:
            break

        print("Processing your question...")
        # These calls are compatible with the updated RAGSystem
        result = rag.query(question)

        print(f"Answer: {result.get('answer', 'No answer')}")
        sources = result.get('sources', [])
        print(f"Sources Found: {len(sources)}")
        if sources:
            print("--- Sources ---")
            for i, source in enumerate(sources):
                print(f"Source {i+1}: {source.get('file_name', 'Unknown File')} (Similarity: {source.get('similarity', 'N/A'):.4f})")
                print(f"Content: {source.get('content', 'N/A')}")
                print("-" * 10)
            print("---------------")


if __name__ == "__main__":
    main()

ValidationError: 1 validation error for CustomOllamaLLM
ollama_url
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

**Reasoning**:
The validation error indicates that the `ollama_url` field is required during initialization, but it's not being passed correctly to the `super().__init__` call in `CustomOllamaLLM`. I need to pass the `ollama_url` and `model_name` to the parent class constructor as part of the `kwargs`.



In [ ]:
from langchain.llms.base import BaseLLM
from langchain_core.outputs import LLMResult
from typing import Any, List, Mapping, Optional
import requests
import asyncio # Import asyncio for potential async operations if needed later

async def query_ollama_api(api_url: str, model_name: str, prompt: str) -> str:
    """Queries the Ollama API for a text generation."""
    url = f"{api_url}/api/generate"
    payload = {
        "model": model_name,
        "prompt": prompt,
        "stream": False
    }
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()  # Raise an exception for bad status codes
        result = response.json()
        return result.get("response", "").strip()
    except requests.exceptions.RequestException as e:
        logger.error(f"Error querying Ollama API: {e}")
        return f"Error querying Ollama API: {e}"


class CustomOllamaLLM(BaseLLM):
    """Custom Langchain LLM for interacting with the Ollama API."""

    ollama_url: str
    model_name: str = "llama3-chatqa"

    @property
    def _llm_type(self) -> str:
        return "custom_ollama"

    def __init__(self, ollama_url: str = "http://localhost:11434", model_name: str = "llama3-chatqa", **kwargs: Any):
        # Pass ollama_url and model_name to the parent class constructor
        super().__init__(ollama_url=ollama_url, model_name=model_name, **kwargs)
        self.ollama_url = ollama_url
        self.model_name = model_name


    # Implement the _generate method as required by the abstract base class
    def _generate(self, prompts: List[str], stop: Optional[List[str]] = None, run_manager: Optional[Any] = None) -> LLMResult:
        """Generate text based on the provided prompts."""
        generations = []
        for prompt in prompts:
            text = self._call(prompt, stop=stop) # Reuse the existing _call logic
            generations.append([{"text": text}]) # Langchain expects a list of lists of generations

        return LLMResult(generations=generations)

    # Keep _call for internal use or backward compatibility if needed, though _generate is the primary method
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """Call the Ollama API to generate text (synchronous helper)."""
        url = f"{self.ollama_url}/api/generate"
        payload = {
            "model": self.model_name,
            "prompt": prompt,
            "stream": False
        }
        try:
            response = requests.post(url, json=payload)
            response.raise_for_status()
            result = response.json()
            return result.get("response", "").strip()
        except requests.exceptions.RequestException as e:
            logger.error(f"Error in CustomOllamaLLM call: {e}")
            return f"Error generating response: {e}"


    @property
    def _identifyingparams(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"ollama_url": self.ollama_url, "model_name": self.model_name}


**Reasoning**:
The previous error was a `ValidationError` because the required fields were not passed to the parent class constructor in `CustomOllamaLLM`. I have corrected the `__init__` method in the `CustomOllamaLLM` class to pass `ollama_url` and `model_name` to `super().__init__`. Now I need to re-run the `main` function code block to verify that this change resolves the validation error and the `RAGSystem` can be initialized.



In [ ]:
# Example usage and testing
def main():
    """Example usage of the RAG system"""
    # Initialize RAG system
    # The RAGSystem class now internally uses CustomOllamaLLM with llama3-chatqa
    rag = RAGSystem(persist_directory="./test_chroma_db")
    rag.initialize()

    # Define your actual documents here
    actual_documents = [
        {
            "file_path": "/content/business_doc.txt", # Replace with your actual file path
        },
        {
            "file_path": "/content/engineering_drawing_specifications.pdf", # Replace with your actual file path
        },
        # Add more documents as needed
        {
            "file_path": "/content/maintenance_job_card.pdf", # Replace with your actual file path
        },
        # Add more documents as needed
        {
            "file_path": "/content/research_doc.txt", # Replace with your actual file path
        },
        # Add more documents as needed
        {
            "file_path": "/content/tech_doc.txt", # Replace with your actual file path
        },
        # Add more documents as needed
    ]

    print("Ingesting actual documents...")
    for doc_info in actual_documents:
        # These calls are compatible with the updated RAGSystem
        result = rag.ingest_document(doc_info["file_path"])
        print(f"Ingested {doc_info['file_path']}: {result}")

    # Show stats
    print(f"\nSystem Stats: {rag.get_stats()}")


    # Interactive querying
    print("\n" + "="*50)
    print("Interactive RAG Querying")
    print("="*50)
    print("Enter your questions. Type 'quit' or 'exit' to stop.")

    while True:
        question = input("\nYour question: ")
        if question.lower() in ["quit", "exit"]:
            break

        print("Processing your question...")
        # These calls are compatible with the updated RAGSystem
        result = rag.query(question)

        print(f"Answer: {result.get('answer', 'No answer')}")
        sources = result.get('sources', [])
        print(f"Sources Found: {len(sources)}")
        if sources:
            print("--- Sources ---")
            for i, source in enumerate(sources):
                print(f"Source {i+1}: {source.get('file_name', 'Unknown File')} (Similarity: {source.get('similarity', 'N/A'):.4f})")
                print(f"Content: {source.get('content', 'N/A')}")
                print("-" * 10)
            print("---------------")


if __name__ == "__main__":
    main()

Ingesting actual documents...
Ingested /content/business_doc.txt: {'status': 'success', 'chunks_created': 1}
Ingested /content/engineering_drawing_specifications.pdf: {'status': 'success', 'chunks_created': 10}
Ingested /content/maintenance_job_card.pdf: {'status': 'success', 'chunks_created': 11}
Ingested /content/research_doc.txt: {'status': 'success', 'chunks_created': 1}
Ingested /content/tech_doc.txt: {'status': 'success', 'chunks_created': 1}

System Stats: {'total_documents': 24}

Interactive RAG Querying
Enter your questions. Type 'quit' or 'exit' to stop.

Your question: tell me about kochi metro
Processing your question...


ERROR:__main__:Error in CustomOllamaLLM call: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7b45d2a56690>: Failed to establish a new connection: [Errno 111] Connection refused'))


Answer: Error generating response: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7b45d2a56690>: Failed to establish a new connection: [Errno 111] Connection refused'))
Sources Found: 5
--- Sources ---
Source 1: maintenance_job_card.pdf (Similarity: 0.1048)
Content: /Info 8 0 R
/Root 7 0 R
/Size 13
>>
startxref
5383
%%EOF
----------
Source 2: engineering_drawing_specifications.pdf (Similarity: 0.1011)
Content: /Info 9 0 R
/Root 8 0 R
/Size 14
>>
startxref
5011
%%EOF
----------
Source 3: maintenance_job_card.pdf (Similarity: 0.0864)
Content: Gat=+D,]1Y&H9tYfL\8k\6.tF2LJ/KKMFmh`(HquiQM%EZEHfClCA&X`]q2L-e[.4,A)l8$ZXmeB:jTq*Is3S_#<C!^i(ZlIg@/&+jD-c]bo0nE0HmkHP1EJ`tK<K%kmFDn*ZU_#sK,@/;5<>Mq3jUb_0LPn.Is:oGLCA2qhH6Q3'H,$DAp[;Z*:HZ='P\8ee*UXK%6[...
----------
Source 4: maintenance_job_card.pdf (Similarity: 0.0760)
Content: %U^k*#?YPCMk#]hfI'/AF2X:^QL#kdiDHgJaQPpYNUa]W!6

## Add provided helper functions

### Subtask:
Include the `query_ollama_api` and `chat_with_ollama` functions in the notebook.


**Reasoning**:
The subtask is to include the `query_ollama_api` and `chat_with_ollama` functions in the notebook. The `query_ollama_api` function is already present in the notebook from a previous step. The `chat_with_ollama` function was mentioned in the original code but not included in the previous steps. I will add the definition for `chat_with_ollama` to the notebook.



In [ ]:
async def chat_with_ollama(api_url: str, model_name: str, messages: List[Dict[str, str]]) -> str:
    """Chats with the Ollama API using the chat endpoint."""
    url = f"{api_url}/api/chat"
    payload = {
        "model": model_name,
        "messages": messages,
        "stream": False
    }
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()  # Raise an exception for bad status codes
        result = response.json()
        return result.get("message", {}).get("content", "").strip()
    except requests.exceptions.RequestException as e:
        logger.error(f"Error chatting with Ollama API: {e}")
        return f"Error chatting with Ollama API: {e}"

## Address connection issues

### Subtask:
Note that the provided code still uses `localhost:11434`, which is unlikely to work directly in Colab. Remind the user that they will need to adjust the URL to point to their accessible Ollama instance.


**Reasoning**:
Add a markdown cell to inform the user about the Ollama API URL configuration.



In [ ]:
### Ollama API URL Configuration

Please note that the current RAG system code is configured to connect to the Ollama API at `http://localhost:11434`.

This address is typically used for a local Ollama installation running on the same machine. When running this notebook in an environment like Google Colab, or if your Ollama instance is running elsewhere, this address will likely not work directly.

**Action Required:**

If you are running Ollama on a different machine or need to access it externally, you must replace `http://localhost:11434` in the code (specifically in the `CustomOllamaLLM` initialization within the `RAGSystem` class) with the actual accessible URL and port of your running Ollama instance.

SyntaxError: invalid syntax (ipython-input-2375610762.py, line 3)

**Reasoning**:
Add a markdown cell with the specified text to inform the user about the Ollama API URL configuration. The previous attempt failed because the markdown content was placed in a Python code block.



In [ ]:
# This is a markdown cell. Please convert this cell to markdown.
"""
### Ollama API URL Configuration

Please note that the current RAG system code is configured to connect to the Ollama API at `http://localhost:11434`.

This address is typically used for a local Ollama installation running on the same machine. When running this notebook in an environment like Google Colab, or if your Ollama instance is running elsewhere, this address will likely not work directly.

**Action Required:**

If you are running Ollama on a different machine or need to access it externally, you must replace `http://localhost:11434` in the code (specifically in the `CustomOllamaLLM` initialization within the `RAGSystem` class) with the actual accessible URL and port of your running Ollama instance.
"""

'\n### Ollama API URL Configuration\n\nPlease note that the current RAG system code is configured to connect to the Ollama API at `http://localhost:11434`.\n\nThis address is typically used for a local Ollama installation running on the same machine. When running this notebook in an environment like Google Colab, or if your Ollama instance is running elsewhere, this address will likely not work directly.\n\n**Action Required:**\n\nIf you are running Ollama on a different machine or need to access it externally, you must replace `http://localhost:11434` in the code (specifically in the `CustomOllamaLLM` initialization within the `RAGSystem` class) with the actual accessible URL and port of your running Ollama instance.\n'

## Summary:

### Data Analysis Key Findings

*   A custom Langchain LLM class (`CustomOllamaLLM`) was successfully created to interface with the Ollama API, specifically targeting the `/api/generate` endpoint.
*   The `CustomOllamaLLM` class was updated to implement both the synchronous `_call` and the required `_generate` methods from the `BaseLLM` abstract class, ensuring compatibility with Langchain's framework.
*   The `RAGSystem` class was modified to initialize and use the `CustomOllamaLLM` with the `llama3-chatqa` model.
*   Helper functions `query_ollama_api` (already present) and `chat_with_ollama` were included in the code.
*   The `main` function's compatibility with the updated `RAGSystem` was confirmed through initialization and simulated usage, although actual query execution depends on a running Ollama instance.
*   A note was added to inform the user about the default `localhost:11434` Ollama URL and the need to change it for different environments (like Colab).

### Insights or Next Steps

*   The system is now configured to use the specified `llama3-chatqa` model via Ollama. The next step is to ensure an Ollama instance with this model is running and accessible at the configured URL for the RAG system to function correctly.
*   Consider enhancing the `CustomOllamaLLM` or `RAGSystem` to gracefully handle Ollama API connection errors or model unavailability, providing clearer feedback to the user.
